<a href="https://colab.research.google.com/github/rahulkumar-datascientist/Neural-Machine-Translation-NMT-with-attention/blob/main/20230113_MSc_DA_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment 2** focuses on the training on a Neural Machine Translation (NMT) system with an attention model.

This is an **individual assignment** and usual rules for plagiarism apply! With this you agree that: "In submitting this work I confirm that it is entirely my own. I acknowledge that I may be invited to online interview if there is any concern in relation to the integrity of my exam." 

**Write comments and documentation.**



## Mounting Drive and setting paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Changing the directory on file to access the files
%cd /content/drive/MyDrive/Colab Notebooks/NLP/Assignment 2

/content/drive/MyDrive/Colab Notebooks/NLP/Assignment 2


## Section 1- Data Collection and Preprocessing 


---


In [ ]:
# Loading necessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model
from numpy import array
from numpy import asarray
from numpy import zeros
import nltk
from nltk import bigrams, trigrams, ngrams
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Activation, dot, concatenate, TimeDistributed

**Task 1  (5 marks)**

---

There are few datasets to train an NMT system available from the OPUS project (http://opus.nlpl.eu/).

*  Download a language pair (preferably European language) and **extract** the file(s) and upload it to CoLab
*  Create a list of lines by splitting the text file at every occurrence accordingly, i.e. source and target language
*  Print number of sentences
*  Limit the number of sentences to 10,000 lines (but more than 5,000 lines)
*  Split the data into train, development and test set
*  Print 100th sentence in original script for source and target language


---
Language Selected: **English - Spanish**

Source: https://opus.nlpl.eu/ 

**Corpus: News-Commentary v16 -- en(English) - es(Spanish)** 

NOTE: Downloaded zip file from moses link -> extracted files and renamed them to English.txt and Spanish.txt 

---

In [ ]:
# SOURCE Language : list of lines
with open("english.txt" , 'r') as english_file:
  source_lines = english_file.read().split('\n')         

# TARGET Language : list of lines
with open("spanish.txt" , 'r') as spanish_file:
  target_lines = spanish_file.read().split('\n')         

# Original number of sentences in SOURCE and TARGET files 
print("Source (English): Total Number of Sentences:", len(source_lines))                        # 49090 sentences
print("Target (Spanish): Total Number of Sentences:", len(target_lines))                        # 49090 sentences

# Fetching only first 10,000 Sentences
source_lines = source_lines[:10000]
target_lines = target_lines[:10000]
print("\nSource (English): Filtered Number of Sentences:", len(source_lines))                   # 10000 sentences                       
print("Target (Spanish): Filtered Number of Sentences:", len(target_lines))                     # 10000 sentences   

# Print 100th sentence in SOURCE and TARGET files
print("\nSource (English): 100th Sentence: ", source_lines[99])
print("Target (Spanish): 100th Sentence: ", target_lines[99])

Source (English): Total Number of Sentences: 49090
Target (Spanish): Total Number of Sentences: 49090

Source (English): Filtered Number of Sentences: 10000
Target (Spanish): Filtered Number of Sentences: 10000

Source (English): 100th Sentence:  This is neither a choice nor a luxury – it is a necessity. 
Target (Spanish): 100th Sentence:  No se trata ni de una opción ni de un lujo: es una necesidad. 


---
**Task 2 (5 marks)** 

---

* Add '<bof\>' to denote beginning of sentence and '<eos\>' to denote the end of the sentence to each target line.
* Perform the pre-processing step of the text.
* Print the last 5 sentences of the preprocessed text.
* Print statistics on the selected dataset:
  * Number of samples
  * Number of unique source language tokens
  * Number of unique target language tokens
  * Max sequence length of source language
  * Max sequence length of target language
  * Source Vocabulary
  * Target Vocabulary





In [ ]:
# function to remove punctuation characters
def punctuations(sentence):
  punct_chars = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

  # remove punctuation from the string
  no_punct = ""
  for char in sentence:
    if char not in punct_chars:
      no_punct = no_punct + char
  
  return(no_punct)

In [ ]:
# Defining the SOURCE & TARGET sets to store the vocabulary 
source_vocab = set()
target_vocab = set()

# Pre-processing the SOURCE and TARGET sentences
# Converting to lowercase
source_lines = [line.lower() for line in source_lines]
target_lines = [line.lower() for line in target_lines]

# removing punctuations
source_lines = [punctuations(line) for line in source_lines]
target_lines = [punctuations(line) for line in target_lines]

# Adding '<bof>' to denote beginning of sentence and '<eos>' to denote the end of the sentence to each target line.
target_lines = ['<bof> ' + target + ' <eos>' for target in target_lines]

# creating the vocab set for SOURCE
for line in source_lines:
  words_per_line = line.split()
  for word in words_per_line:
    if word not in source_vocab:
      source_vocab.add(word)

# creating the vocab set for TARGET
for line in target_lines:
  words_per_line = line.split()
  for word in words_per_line:
    if word not in target_vocab:
      target_vocab.add(word)

In [ ]:
# Last 5 sentences after preprocessing
# SOURCE lines
print("Last 5 sentences from preprocessed SOURCE")
i = 4
for line in source_lines[-5:]:
  print("sentence ", len(source_lines)-i, ": ", line)
  i-=1

# TARGET lines
print("\nLast 5 sentences from preprocessed TARGET")
i = 4
for line in target_lines[-5:]:
  print("sentence ", len(target_lines)-i, ": ", line)
  i-=1 

Last 5 sentences from preprocessed SOURCE
sentence  9996 :  the report was as scathing as it was scientifically clear 
sentence  9997 :  despite many past promises of a cleanup ogoniland remains in environmental agony impoverished and sickened by the oil industry 
sentence  9998 :  unep also offered clear and detailed recommendations including emergency measures to ensure safe drinking water cleanup activities targeting the mangroves and soils publichealth studies to identify and counteract the consequences of pollution and a new regulatory framework 
sentence  9999 :  the world’s governments have recently agreed to move to a new framework for sustainable development declaring their intention to adopt sustainable development goals at the rio+20 summit in june 
sentence  10000 :  the sdgs offer a critical opportunity for the world to set clear compelling standards for government and corporate behavior 

Last 5 sentences from preprocessed TARGET
sentence  9996 :  <bof> el informe era tan

In [ ]:
# Statistics of the dataset -- Reference Lab 8
# SOURCE and TARGET vocabulary as sorted list 
source_vocab = sorted(list(source_vocab))
target_vocab = sorted(list(target_vocab))

# Number of Encoder and Decoder tokens
len_source_vocab = len(source_vocab)
len_target_vocab = len(target_vocab)

# Max sequence length for Encoder and Decoder
max_seq_len_source = max([len(txt) for txt in source_lines])
max_seq_len_target = max([len(txt) for txt in target_lines])

print('Number of samples\t\t\t:', len(source_lines))
print('Number of unique SOURCE language tokens\t:', len_source_vocab)
print('Number of unique TARGET language tokens\t:', len_target_vocab)
print('Max sequence length of SOURCE language\t:', max_seq_len_source)
print('Max sequence length of TARGET language\t:', max_seq_len_target)
print("Source Vocabulary\t\t\t:",source_vocab)
print("Target Vocabulary\t\t\t:",target_vocab)

Number of samples			: 10000
Number of unique SOURCE language tokens	: 17734
Number of unique TARGET language tokens	: 21650
Max sequence length of SOURCE language	: 660
Max sequence length of TARGET language	: 759
Source Vocabulary			: ['0', '001', '001kilowatthour', '005kilowatthour', '006', '006gallon', '006kilowatthour', '007kilowatthour', '009kilowatthour', '01', '010kilowatthour', '011', '011kilowatthour', '016', '016kilowatthour', '02', '03', '04', '05', '06°c', '07', '07ofgdp', '08', '08”', '1', '10', '100', '1000', '10000', '100000', '100150', '100th', '101', '1015', '1016page', '102', '106', '108', '10city', '11', '1100', '1110', '112', '115', '11th', '11year', '12', '120', '1200', '123', '125', '1255', '126', '129', '12th', '13', '1300', '135', '1373', '13th', '14', '140', '145', '1461', '14th', '14year', '14€1', '15', '150', '1500', '15000', '150year', '1520', '155', '1550', '158', '159', '15year', '16', '160', '16000', '1644', '17', '1703', '175', '1787', '1789', '18', '180

**Task 3 (15 marks)** 

---

*  Assign each unique word an integer value (5 marks).
*  Create word embedding for your vocabulary using pre-trained embeddings, for example GloVe or fastText (10 marks) (https://nlp.stanford.edu/projects/glove/ , https://fasttext.cc/docs/en/english-vectors.html)
* Print the first line of the embeddings (see below) 

In [ ]:
# Assigning each word/token an integer value for both SOURCE and TARGET 
source_vocab_index = dict([(word,i) for i, word in enumerate(source_vocab)])                  # 17734 tokens --- same as above
target_vocab_index = dict([(word,i) for i, word in enumerate(target_vocab)])                  # 21650 tokens --- same as above

Tried using various fastText word-embedding(smallest: "wiki-news-300d-1M.vec.zip") but the session always crashed after using all the available RAM on google colab.

Thus using GloVe word-embedding --- Reference from Lab 2

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

--2021-05-03 21:15:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-05-03 21:15:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-05-03 21:15:58--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
path_to_glove_file = "glove.6B.300d.txt"

embeddings = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings[word] = coefs

In [ ]:
# Creating our own word-embedding from the pre-trained GloVe embedding

# Embedding Shape - 300 - because the pretrained model has 300 dimensions
# embeddings['the'].shape

source_embedding = zeros((len(source_vocab_index), 300))

for word, index in source_vocab_index.items():
  try:
    vector = embeddings[word]
    source_embedding[index] = vector
  except:
    continue

In [ ]:
# Printing First line of embedding
print(source_embedding[0])

[ 1.89700007e-01  7.03390002e-01  1.54339999e-01  4.21669990e-01
 -5.38580000e-01  3.33579987e-01 -1.73299998e-01  5.42779982e-01
  1.96180001e-01 -2.33419999e-01  1.28680003e+00  7.83479989e-01
 -4.58469987e-01  6.78990036e-02 -9.15229991e-02  1.57550007e-01
 -2.81749994e-01 -2.54729986e-01 -4.46799994e-01 -8.54499996e-01
 -7.90170014e-01 -8.75220001e-01 -5.38029987e-03 -1.73260003e-01
  9.88710001e-02 -1.35089993e-01  1.04120001e-01 -2.31079996e-01
 -6.50659978e-01 -5.49149990e-01  3.52200009e-02  6.41539991e-01
 -5.41400015e-01  1.96470007e-01 -1.34270000e+00 -5.15910015e-02
  5.32649994e-01  8.40380013e-01 -6.05379999e-01 -2.61090010e-01
  5.88400006e-01  1.64910004e-01 -1.17340004e+00  3.61350000e-01
 -4.09350008e-01 -1.80069998e-01  3.11809987e-01  1.65610000e-01
 -1.53630003e-01  3.46179992e-01 -2.55650014e-01 -1.90540001e-01
 -1.16899997e-01  2.46509999e-01  8.36519971e-02 -1.83850005e-01
  2.65980005e-01 -3.34060013e-01 -4.99550020e-03  4.11379993e-01
 -2.77960002e-01 -2.82480

## Section 2 Translation Model training



---



**Task 4 (15 marks)**
* Provide code for the encoder using Keras LSTM (5 marks)
* Provide code for the decoder using Keras LSTM (5 marks)
* Train the sequence2sequence (encoder-decoder) model (5 marks) 


In [ ]:
# encoder code goes here

In [ ]:
# decoder code goes here

In [ ]:
model= # your code goes here
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.summary()

In [ ]:
model.fit(#your code goes here)  
model.save('seq2seq_source_target.h5')

## Section 3 Testing

---

**Task 5 (20 marks)**

* Use the trained model to translate the text from the source into the target language (10 marks). 
* Use the test/evaluation set (see Section 1) and perform an automatic evaluation with the BLEU metric (10 marks). 
You can use the NLTK library to calculate BLEU.

In [ ]:
#Your code goes here

# Section 4 Attention

---



**Task 5 (40 Marks)** Sequence2Sequence

* Extend the existing Seq2Seq model with an attention mechanism [Discussed in Class]
* Create sequence2sequence model with attention (15 marks)
* Train the model with the same data from Section 1 (10 marks)
* Translate the evaluation set using the sequence2sequence attention model (10 marks)
* Evaluate the translations made with the sequence2sequence attention model and compare it with the model without attention using BLEU (5 marks)

In [ ]:
#Your code goes here